In [50]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
trends = json.load(open("../trends-data/processed_trends.json"))

In [3]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

c:\Users\diasrodr\Anaconda3\envs\webscrap\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\diasrodr\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [30]:
trends[list(trends.keys())[0]]

['Days until SpiderManAcrossTheSpiderVerse releases ',
 'More explosive 2D FX I animated from the recent SpiderManAcrossTheSpiderVerse trailers Out very soon Spiderman2099 2DFX ',
 'SonyPicturesUK So looking forward to this SpiderManAcrossTheSpiderVerse spiderverse',
 'SpiderMan Across the SpiderVerse Trailer  SpidermanAcrossTheSpiderverse Spiderman Marvel Sony Trailer',
 'When you have enough points to get the the Future Sense colllectors box for free Just pay shipping GFUEL SpiderMan SpiderManAcrossTheSpiderVerse GFuelEnergy GammaLabs ',
 'SpiderManAcrossTheSpiderVerse will end on a great cliffhanger I was very satisfied after The Empire Strikes Back And hopefully This is our Empire Joaquim Dos Santos CoDirector ',
 'deleted trailer shows new look at Ben Reilly aka the Scarlet Spider AcrossTheSpiderVerse SpiderManAcrossTheSpiderVerse SpiderVerse leak scarletspider benreilly itsv atsv intothespiderverse MilesMorales 3CFilmss SpiderLeaks ',
 'PENI PARKER APPEAR IN ACROSS THE SPIDERVERS

In [6]:
list(trends.keys())[0]

'#SpiderManAcrossTheSpiderVerse'

In [8]:
treds_entities = []
for trend_topic in list(trends.keys()):
    for trend in trends[trend_topic]:
        entities = nlp(trend)
        for ent in entities:
            treds_entities.append({"trend":trend_topic, "word":ent["word"], "entity":ent["entity"]})

In [4]:
treds_entities = []
for trend in trends[list(trends.keys())[0]]:
    entities = nlp(trend)
    for ent in entities:
        treds_entities.append({"word":ent["word"], "entity":ent["entity"]})

In [17]:
df.entity.value_counts()

I-ORG     14656
B-PER      8956
I-PER      8945
B-ORG      6414
I-MISC     4463
B-MISC     2234
B-LOC      1399
I-LOC      1089
Name: entity, dtype: int64

In [48]:
df = pd.DataFrame(treds_entities)#.groupby("trend")["word"].apply(set)
df_for_pinterest = df[((df.entity == "I-PER") | (df.entity == "B-PER")) & ~(df.word.astype(str).str.startswith('#'))]
# df_for_pinterest["pinterest"] = df_for_pinterest.word + " " + df_for_pinterest.trend
df_for_pinterest.groupby(["trend","word"])[["word"]].count()
df_for_pinterest.groupby(["trend",'word']).agg({"word":"count"}).rename(columns={'word':'count'}).reset_index().groupby(["trend"])["word","count"].max("count")
df_for_pinterest.groupby(['trend'])['word'].apply(lambda x: x.value_counts().index[0]).reset_index()

C:\Users\diasrodr\AppData\Local\Temp\ipykernel_22732\2727526669.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_for_pinterest.groupby(["trend",'word']).agg({"word":"count"}).rename(columns={'word':'count'}).reset_index().groupby(["trend"])["word","count"].max("count")


,trend,word
0,#911LoneStar,Ta
1,#911onFOX,Buck
2,#AdamKutnerPowerPlay,Adam
3,#AgustD_SUGA_Tour_in_LA,A
4,#AllForCITY,Matt
...,...,...
550,hanbin,De
551,jack antonoff,Matt
552,lorde,Sasha
553,luke hemmings,Luke


### 3 palavras com mais frequência

In [86]:
vectorizer = TfidfVectorizer()

vectors = vectorizer.fit_transform(raw_documents=trends['#911LoneStar'])
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [87]:
global_top10_idx = df.max(axis=0).argsort()[:3]
np.asarray(feature_names)[global_top10_idx]

array(['youve', 'girl', 'got'], dtype=object)